# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120738e+02     1.916022e+00
 * time: 0.4219391345977783
     1     1.111326e+01     8.705235e-01
 * time: 1.6427311897277832
     2    -1.133160e+01     1.020552e+00
 * time: 1.7145600318908691
     3    -3.372012e+01     7.611188e-01
 * time: 1.8152000904083252
     4    -4.727798e+01     5.578028e-01
 * time: 1.9361331462860107
     5    -5.684817e+01     1.892117e-01
 * time: 2.018908977508545
     6    -5.982954e+01     1.305454e-01
 * time: 2.0805912017822266
     7    -6.093841e+01     5.401616e-02
 * time: 2.1393861770629883
     8    -6.135900e+01     5.672253e-02
 * time: 2.204686164855957
     9    -6.164820e+01     4.612632e-02
 * time: 2.285731077194214
    10    -6.186473e+01     2.457453e-02
 * time: 2.347414016723633
    11    -6.202573e+01     1.877065e-02
 * time: 2.408764123916626
    12    -6.208857e+01     1.465363e-02
 * time: 2.472053050994873
    13    -6.215173e+01     1.426750e-02
 * time: 2.539340019226074


In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671069
    AtomicLocal         -18.8557655
    AtomicNonlocal      14.8522623
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336820

    total               -62.261666460444